In [1]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from tqdm import tqdm

import matplotlib.pyplot as plt

from ad_config_search.utils import (
    get_detection_tracking_accuracies, 
    get_binned_array, 
    ray_map_np, 
    split_run, 
    get_fn_without_ext, 
    get_rows, 
    condense_moment_policy_scores, 
    compare_policy_scores, 
    get_gt_and_numerator,
    compute_mota_scores,
    make_data
)
from ad_config_search.config_space_control import ConfigSpace

%load_ext autoreload
%autoreload 2

In [2]:
loc1 = list(set(Path("../../ad-config-search/12-02-results/").glob("**/*.csv")) - set(list(Path("../../ad-config-search/12-02-results/").glob("**/ray_results/**/*.csv"))))  # removing 4 folders which contain repeats
loc2 = list(Path("../../ad-config-search/12-03-v2/").glob("**/*.csv"))
loc3 = list(Path("../../ad-config-search/12-06-v8/").glob("**/*.csv"))
loc4 = list(Path("../../ad-config-search/12-06-v7/").glob("**/*.csv"))

In [3]:
all_csvs = loc1 + loc2 + loc3 + loc4

In [11]:
import ray

In [12]:
ray.shutdown()
ray.init()

2021-02-14 06:09:51,899	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '169.229.48.125',
 'raylet_ip_address': '169.229.48.125',
 'redis_address': '169.229.48.125:60785',
 'object_store_address': '/tmp/ray/session_2021-02-14_06-09-51_279232_95337/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-14_06-09-51_279232_95337/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-02-14_06-09-51_279232_95337',
 'metrics_export_port': 61851,
 'node_id': '365ebf0c41ddb76cd8c5128e010580a4cc1cf026'}

In [16]:
num_gt_matrix = ray_map_np(all_csvs, get_gt_and_numerator, 120)

100%|██████████| 3788/3788 [05:48<00:00, 10.86it/s]


In [17]:
num_gt_matrix = np.squeeze(num_gt_matrix)

In [18]:
num_gt_matrix.shape

(454656, 2, 200)

In [19]:
num_gt_matrix

array([[[nan,  4.,  0., ...,  2.,  0.,  0.],
        [nan,  4.,  0., ...,  2.,  0.,  0.]],

       [[nan, 14., 14., ..., 22., 18.,  0.],
        [nan, 14., 14., ..., 33., 31.,  0.]],

       [[nan,  5.,  0., ...,  3.,  0.,  0.],
        [nan,  5.,  0., ..., 10.,  0.,  0.]],

       ...,

       [[nan, nan, 27., ..., 30.,  0.,  0.],
        [nan, nan, 34., ..., 37.,  0.,  0.]],

       [[nan, 58., 58., ..., 41.,  0.,  0.],
        [nan, 58., 58., ..., 44.,  0.,  0.]],

       [[nan,  9.,  9., ...,  9.,  8.,  0.],
        [nan,  9.,  9., ...,  5.,  4.,  0.]]])

In [58]:
np.save("num_gt_matrix", num_gt_matrix)

In [20]:
# We add an index column because pandas does not guarantee that the order of rows will be the same as the order of the row dicts passed here
df_configs = pd.DataFrame(
    [{**split_run(get_fn_without_ext(x)), "index": i} for i, x in enumerate(all_csvs)]
)

In [21]:
df_configs = df_configs.drop(columns=["T-ignore-len"])
df_configs = df_configs.drop(columns=["res-h"])
df_configs = df_configs.drop(columns=["res-w"])
df_configs = df_configs.drop(columns=["dataset"])

In [22]:
df_configs = df_configs.loc[df_configs["D-seq-pol"] != "infinite"]
df_configs["T-max-age"] = df_configs["T-max-age"].astype(int)
df_configs["T-every-nth-det"] = df_configs["T-every-nth-det"].astype(int)
df_configs["T-min-iou"] = df_configs["T-min-iou"].astype(float)
df_configs["D-conf"] = df_configs["D-conf"].astype(float)
df_configs["D-seq-pol"] = df_configs["D-seq-pol"].astype("category")
df_configs["D-model"] = df_configs["D-model"].astype("category")
df_configs = df_configs.set_index("index")

In [23]:
df_configs

,run,D-model,D-conf,D-seq-pol,T-model,T-min-iou,T-max-age,T-every-nth-det
index,,,,,,,,
0,training_0000-S3,efficientdet-d3,0.3,wait,sort,0.5,1,1
1,training_0000-S17,efficientdet-d5,0.3,wait,sort,0.3,7,1
3,training_0000-S5,efficientdet-d5,0.5,eager,sort,0.5,1,3
4,training_0000-S5,efficientdet-d3,0.3,eager,sort,0.3,1,1
5,training_0000-S21,efficientdet-d5,0.5,tail-aware,sort,0.2,1,3
...,...,...,...,...,...,...,...,...
454650,training_0003-S0,efficientdet-d2,0.3,wait,sort,0.2,5,3
454651,training_0003-S20,efficientdet-d6,0.3,wait,sort,0.2,5,1
454652,training_0002-S8,efficientdet-d4,0.3,tail-aware,sort,0.5,5,5


In [61]:
with open("num_gt_matrix_index.pl", 'wb') as f:
    pickle.dump(df_configs, f)